In [1]:
!pip install google-play-scraper

In [2]:
!pip install Sastrawi

In [3]:
!pip install emoji

In [4]:
from google_play_scraper import reviews
import pandas as pd
from google.colab import files
import csv
import string
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import emoji
import pandas as pd
import numpy as np
import re

In [5]:
nltk.download('stopwords')
stop_words = set(stopwords.words('indonesian'))

nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
result, _ = reviews(
    'com.naver.linewebtoon',
    lang='id',
    country='id',
    count=15000,
    filter_score_with=None
)

for r in result:
    print(r['userName'], r['score'], r['content'])

Output streaming akan dipotong hingga 5000 baris terakhir.
Pengguna Google 5 cerita nya sangat menarik dan bagus
Pengguna Google 1 Hadeewuuh... Update komiknya jauh ketinggalan.. sama web2 gratis..
Pengguna Google 5 gua suka banget sih njay pokonya seru
Pengguna Google 5 nice
Pengguna Google 5 bagus
Pengguna Google 5 Baik aja
Pengguna Google 4 Serial nya seru banget tapi sayang nya ada yg gak bisa baca ada jgk harus pakek iklan tapi seru jgk👍👍👍👍👍👍👍👍👍
Pengguna Google 5 mantap
Pengguna Google 1 Kok jadi upload yang 18+. Verifikasi umur tinggal klik tanpa bukti apapun. Walaupun ada adegan yang dipotong, tapi itu jelas kasih liat adegan/obrolan dewasa. Padahal yang baca ada anak-anak di bawah umur.
Pengguna Google 1 apk ini bagus banget,tapi kenapa aku ngasi bintang 1? Aku mau baca siyoon lagi gabisa bjr,katanya 18+ dan mengandung konten dewasa...😭
Pengguna Google 5 seru banget tapi tolong mudah kan masuk mode Inggris soal nya lebih suka komik Inggris
Pengguna Google 5 keren
Pengguna Googl

In [7]:
df = pd.DataFrame(result)
df.to_csv("dataset_scraping.csv", index=False)

In [8]:
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,853ca217-fab6-4f86-b251-d99ee086c0da,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,aplikasi nya bagus tapi kenapa GK bisa buka ga...,2,0,None,2025-04-07 04:44:39,None,NaT,None
1,762d9579-6161-4799-9a8f-bd3760165e4c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,perbaiki bug inii plss setiap mau login itu di...,3,0,3.4.1,2025-04-07 04:26:34,None,NaT,3.4.1
2,cad1ed86-5da3-4501-ad5f-7faa80b23403,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus banget suka,5,0,3.6.2,2025-04-07 04:06:02,None,NaT,3.6.2
3,4e2d6230-7c26-44a2-815d-a4e479c79b3f,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,bagus bangettt,5,0,3.6.2,2025-04-07 03:20:39,None,NaT,3.6.2
4,9c122068-b2b3-45cd-b108-024840765422,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Minn kapan ad genre BL. Kalu ad pasti rame. Ay...,5,0,None,2025-04-07 03:20:32,None,NaT,None


In [9]:
print(f"Jumlah review: {len(df)}")

Jumlah review: 15000


In [10]:
# files.download('dataset_scraping.csv')

Preprosesing Text

In [11]:
clean_df = df.copy()

In [12]:
clean_df = clean_df.drop(columns=['userImage', 'replyContent', 'repliedAt'])
clean_df = clean_df.drop_duplicates(subset='content')
clean_df = clean_df.dropna(subset=['content', 'score'])

In [13]:
# Reset index setelah pembersihan
clean_df = clean_df.reset_index(drop=True)

print(f"Jumlah data akhir: {clean_df.shape[0]} baris dan {clean_df.shape[1]} kolom")
clean_df.head()

Jumlah data akhir: 11148 baris dan 8 kolom


,reviewId,userName,content,score,thumbsUpCount,reviewCreatedVersion,at,appVersion
0,853ca217-fab6-4f86-b251-d99ee086c0da,Pengguna Google,aplikasi nya bagus tapi kenapa GK bisa buka ga...,2,0,None,2025-04-07 04:44:39,None
1,762d9579-6161-4799-9a8f-bd3760165e4c,Pengguna Google,perbaiki bug inii plss setiap mau login itu di...,3,0,3.4.1,2025-04-07 04:26:34,3.4.1
2,cad1ed86-5da3-4501-ad5f-7faa80b23403,Pengguna Google,bagus banget suka,5,0,3.6.2,2025-04-07 04:06:02,3.6.2
3,4e2d6230-7c26-44a2-815d-a4e479c79b3f,Pengguna Google,bagus bangettt,5,0,3.6.2,2025-04-07 03:20:39,3.6.2
4,9c122068-b2b3-45cd-b108-024840765422,Pengguna Google,Minn kapan ad genre BL. Kalu ad pasti rame. Ay...,5,0,None,2025-04-07 03:20:32,None


In [14]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = emoji.replace_emoji(text, replace='')
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = text.strip()
    return text

In [15]:
def casefoldingText(text):
     text = text.lower()
     return text

In [16]:
# Perbaikan kata gaul dan singkatan
slangwords = {
     "@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "bisa":"dapat","bags ":"bagus","bgs":"bagus",
    "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal","gak":"tidak","+":"baik","-":"buruk",
    "ga": "tidak", "gak":"tidak", "nggak":"tidak","enggak":"tidak", "yaa": "ya", "aku": "saya", "ku": "saya","lu": "kamu", "loe": "kamu",
    "guys":"teman - teman","kamu": "anda", "bisa": "dapat", "gila": "hebat", 'udah':'sudah', "ges":"teman - teman",
    "gue": "saya", "gw": "saya", "begetee": "banget", "gampang": "mudah","seru":"asik", "lop":"suka",
     "tp":'tapi', "tpi":"tapi", "gg":"keren", "seru":"asik", "trs":"terus" , "donlod":"unduh","mls":'males',
     "yng":"yang","yg":"yang","gg":"keren", "lg":"lagi", "dpt":"dapat", "krg":"kurang","anjir": "anjir", "anjay": "anjay",
     "sukak":"suka", "mantap":"hebat", "q":"saya","bjier":"astaga", "bgs":"bagus","bosen":"bosan","rekomen":"rekomendasi",
     "gwe":"saya", "ttp":"tetap", "aj":"aja", "/":"atau","menyala":"keren","bet":"banget","btw": "ngomong-ngomong",
     "gegara":"karena","pakek":"pakai","B aja":"biasa", "mayan":"lumayan","mewing":"keren","cuan": "untung",
     "sikibidi":"keren", "gud":"good", "klau":"jika", "mantul":"keren","happy":"senang", "plis":"tolong","app":"aplikasi","apk":"aplikasi"

}

def fix_slangwords(text):
    words = text.split()
    fixed_words = [slangwords.get(word.lower(), word) for word in words]
    return ' '.join(fixed_words)

In [17]:
# Hapus lebih dari 2 huruf berdekatan
def remove_double_letters(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [18]:
# Menangani Typo
def correct_typos_and_slang(text):
    text = remove_double_letters(text)
    text = fix_slangwords(text)
    text = casefoldingText(text)

    return text

In [19]:
def DoubleMeans_Text(text):
    # Ubah pola seperti kata^2 menjadi kata kata
    text = re.sub(r'(\w+)\^2', r'\1 \1', text)
    return text

In [20]:
# Tokenisasi
def tokenizingText(text):
    return word_tokenize(text)

In [21]:
# Stopword removal
def filteringText(tokens, level="default"):
    stopwords_ind = set(stopwords.words('indonesian'))
    stopwords_eng = set(stopwords.words('english'))

    tambahan_default = {'iya', 'ya', 'sih', 'kan', "loh", "kah", "woi", "woy", 'deh','dah','anu', 'plis', "cuy","bro", "coy","weh","nge",'ngtd'}
    tambahan_agresif = tambahan_default.union({'bezir', 'bezier', 'bejir','wanjay','wow','wew','woah','wah'})
    if level == "aggressive":
        all_stopwords = stopwords_ind.union(stopwords_eng).union(tambahan_agresif)
    else:
        all_stopwords = stopwords_ind.union(stopwords_eng).union(tambahan_default)
    return [word for word in tokens if word not in all_stopwords]

In [22]:
# Stemming Bahasa Indonesia
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmingText(text):
    words = text.split()
    stemmed = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed)

In [23]:
# Konvert token list menjadi kalimat
def toSentence(tokens):
    if isinstance(tokens, list):
        return ' '.join(tokens)
    else:
        return ''

In [24]:
# Pipeline penerapan preprocessing
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)
clean_df['text_corrected'] = clean_df['text_clean'].apply(correct_typos_and_slang)
clean_df['text_DoubleMeans'] = clean_df['text_corrected'].apply(DoubleMeans_Text)
clean_df['text_token'] = clean_df['text_DoubleMeans'].apply(tokenizingText)
clean_df['text_nostop'] = clean_df['text_token'].apply(filteringText)
clean_df['text_sentence'] = clean_df['text_nostop'].apply(toSentence)
clean_df['text_final'] = clean_df['text_sentence'].apply(stemmingText)

clean_df[['content', 'text_final']].head(15)

,content,text_final
0,aplikasi nya bagus tapi kenapa GK bisa buka ga...,aplikasi nya bagus gk buka game dapet koin tem...
1,perbaiki bug inii plss setiap mau login itu di...,baik bug pls login loading bngt pas udh login ...
2,bagus banget suka,bagus banget suka
3,bagus bangettt,bagus banget
4,Minn kapan ad genre BL. Kalu ad pasti rame. Ay...,min ad genre bl kalu ad rame ayo min genre bl
5,cerita yang sangat penting untuk diperhatikan ...,cerita perhati hidup contoh orang laku sekolah...
6,seru,asik
7,"Enak si untuk liat cerita, bagus deh",enak si liat cerita bagus
8,aku sukaa bgtt sama apk ini tpiii apk ini tu g...,suka banget aplikasi aplikasi tu sengaja hapus...
9,bagus banget cerita nya aku suka,bagus banget cerita nya suka


In [26]:
# Save the final cleaned dataset
clean_df.to_csv('cleaned_dataset_uji.csv', index=False)
files.download('cleaned_dataset_uji.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>